In [ ]:
%pip install sagemaker_ssh_helper sagemaker

In [ ]:
import os

ACCOUNT_ID=os.popen("aws sts get-caller-identity --query 'Account' --output text").read().strip()
ROLE_NAME = 'sshhelper-for-sagemaker-role'
ROLE_ARN = f"arn:aws:iam::{ACCOUNT_ID}:role/{ROLE_NAME}"

import logging
from sagemaker.pytorch import PyTorch
from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper  # <--NEW--


estimator = PyTorch(
    entry_point='fake-train.py',
    source_dir='src/',
    dependencies=[SSHEstimatorWrapper.dependency_dir()],  # <--NEW 
    # (alternatively, add sagemaker_ssh_helper into requirements.txt 
    # inside source dir) --
    role=ROLE_ARN,
    framework_version='1.9.1',
    py_version='py38',
    instance_count=1,
    instance_type='ml.m5.xlarge',
)

ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=600)  # <--NEW--

estimator.fit(wait=False)

In [ ]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=900)  # <--NEW-- 

In [ ]:
logging.info(f"To connect over SSH run: sm-local-ssh-training connect {ssh_wrapper.training_job_name()}")

In [ ]:
logging.info(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")